# Всё, что нужно знать о декораторах Python

В этом руководстве по декораторам мы рассмотрим, что они собой представляют, как их создавать и использовать. По определению, декоратор – это функция, которая принимает другую функцию и расширяет поведение последней, не изменяя ее явным образом. В этом туториале мы постараемся понять, что это значит и как реализуется.

Текст Jupyter-блокнота является сокращенным переводом публикации Гейра Арне Хьелле [Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/). Код из оригинальной статьи доступен в [GitHub-репозитории](https://github.com/realpython/materials/tree/master/primer-on-python-decorators).

Так как в коде много примеров, я также сделал этот Jupyter-блокнот с текстом перевода и адаптированным кодом, чтобы его было проще запускать интерактивно. Текст также опубликован на сайте [proglib.io](https://proglib.io/).

# 1. Предварительные соображения: функции

Прежде чем начать разбираться в декораторах, немного поговорим о важных для их понимания свойствах функций.

## 1.1. Передача функции в качестве аргумента

В Python функции можно передавать и использовать в качестве аргументов, как и любой другой объект. Рассмотрим следующие три функции:

In [ ]:
def say_hello(name):
    return f"Привет, {name}!"

def be_awesome(name):
    return f"Класс, {name}, быть вместе так круто!"

def greet_vanya(greeter_func):
    return greeter_func("Ваня")

Здесь `say_hello()` и `be_awesome()` – обычные функции, которые получают строковую переменную `name`. Функция `greet_bob()` в качестве аргумента получает другую функцию, например `say_hello()` или `be_awesome()`:

In [ ]:
greet_vanya(say_hello)

In [ ]:
greet_vanya(be_awesome)

При передаче в качестве аргумента имя функции указывается без скобок – передаётся только ссылка на функцию. Сама функция не выполняется, пока не будет вызвана функция `greet_vanya()`.

## 1.2. Внутренние функции
Функции, определенные внутри других функций, называются внутренними (inner functions). Пример функции с двумя внутренними функциями:

In [ ]:
def parent():
    print("Привет из функции parent().")

    def first_child():
        print("Привет из функции first_child().")

    def second_child():
        print("Привет из функции second_child().")

    second_child()
    first_child()

Что произойдёт при вызове функции `parent()`? Остановитесь, чтобы подумать. Вывод будет следующим:

In [ ]:
parent()

Обратите внимание, что порядок, в котором определены внутренние функции, не имеет значения. Печать происходит только при вызове функции.

Внутренние функции не определены, пока не вызвана родительская функция. То есть они локально ограничены `parent()` и существуют только внутри нее, как локальные переменные. При вызове функции `first_child()` за пределами `parent()` будет получена ошибка:

In [ ]:
first_child()

## 1.3. Возврат функций из функций
Python позволяет использовать функции в качестве возвращаемых значений. В следующем примере возвращается одна из внутренних функций внешней функции `parent()`:

In [ ]:
def parent(num):
    def first_child():
        return "Привет, меня зовут Ксавье."

    def second_child():
        return "Зови меня X Æ A-12."

    if num == 1:
        return first_child
    else:
        return second_child

В инструкции `return` возвращается ссылка на функцию, то есть имя функции указывается без скобок (иначе бы возвращался результат выполнения функции).

In [ ]:
first = parent(1)
second = parent(2)
first

In [ ]:
second

В приведенном примере `first` и `second` – переменные, в которые были записаны ссылки на локальные функции `first_child()` и `second_child()` внутри функции `parent()`. Теперь `first` и `second` можно использовать как обычные функции, хотя функции, на которые они указывают, недоступны напрямую:

In [ ]:
first()

In [ ]:
second()

Обратите внимание, что в предыдущем разделе о внутренних функциях мы не имели доступа к `first_child()`. В последнем же примере мы получили ссылку на каждую функцию и можем их вызывать в будущем.

# 2. Простые декораторы 💅
## 2.1. Общая идея: используем знания о функциях
Теперь, когда мы увидели, что функции в Python похожи на любые другие объекты, нам будет проще понять «магию» декораторов. Начнём с искусственного примера, поясняющего идею:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("До вызова функции.")
        func()
        print("После вызова функции.")
    return wrapper

def say_whee():
    print("Ура!")

say_whee = my_decorator(say_whee)

Знаете, что произойдёт при вызове `say_where()`?

In [ ]:
say_whee()

Чтобы понять, что происходит, оглянемся на предыдущие примеры. Мы просто применяем всё, что узнали до сих пор. Декорирование происходит в последней строчке:

`say_whee = my_decorator(say_whee)`

Мы передаем в функцию `my_decorator()` ссылку на функцию say_whee. В `my_decorator()` есть внутренняя функция `wrapper()`, ссылка на которую возвращается в инструкции `return` внешней функции. В результате мы передали в `my_decorator()` в качестве аргумента ссылку на одну функцию, а назад получили ссылку на её функцию-обёртку.

Теперь имя `say_whee` указывает на внутреннюю функцию `wrapper`:

In [ ]:
say_whee

Однако `wrapper()` содержит ссылку на оригинал `say_whee()` и вызывает эту функцию между двумя вызовами `print()`. Проще говоря, декоратор обертывает функцию, изменяя ее поведение.

**Добавим динамики**. Рассмотрим второй пример, иллюстрирующий динамическое поведение декораторов. Сделаем так, чтобы наша функция кричала "Ура!" только в дневное время.

In [ ]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 8 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Тише, соседи спят!
    return wrapper

def say_whee():
    print("Ура!")

say_whee = not_during_the_night(say_whee)

Декорированная функция `say_whee()` будет выводить `"Ура"` только, если она запущена в интервале c 8:00 до 22:00 (чтобы проверить разницу в поведении, «подкрутите стрелки» ⏰).

## 2.2. Немного синтаксического сахара! 🍭
То, как мы декорировали `say_whee()`, прямо скажем, выглядит неуклюже. В последнем примере мы три раза использовали имя say_whee: при определении функции-оригинала, при передаче ссылку в функцию `not_during_the_night()` и при переопределении имени для создания ссылки на декоратор.

Чтобы не заниматься такими глупостями, в Python можно создать декоратор с помощью символа `@`. Следующий код эквивалентен первому рассмотренному примеру:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("До вызова функции.")
        func()
        print("После вызова функции.")
    return wrapper

@my_decorator
def say_whee():
    print("Ура!")

In [ ]:
say_whee()

То есть инструкция `@my_decorator`, идущая перед определением функции `say_whee()` эквивалентна инструкции `say_whee = my_decorator(say_whee)`.

## 2.3. Повторное использование декораторов
Как и любую другую функцию, декоратор можно поместить в отдельный модуль и использовать для различных целей. К примеру, можно создать файл `decorators.py` со всеми декораторами и импортировать из него необходимые функции. 

В отличие от статей, на которые мы сослались в начале документа, для удобства работы мы будем описывать и переопределеять все декораторы непосредственном в этом Jupyter-блоноте.

## 2.4. Декорирование функций, принимающих аргументы 📥

Пусть у нас есть функция, принимающая аргументы. Можем ли мы ее декорировать? Попробуем. Напишем простой декоратор, который дважды выполняет переданную функцию:

In [ ]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Привет, {name}!")

К сожалению, запуск кода вызовет ошибку:

In [ ]:
greet("мир")

Проблема в том, что внутренняя функция декоратора `wrapper_do_twice()` не принимает аргументов. Нужно добавить их обработку. Перепишем декоратор следующим образом:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Привет, {name}!")

Теперь внутренняя функция декоратора принимает любое число документов и пересылает их декорируемой функции. Так обе декорированные функции будут работать корректно:

In [ ]:
say_whee()

In [ ]:
greet("мир")

## 2.5. Возвращение значение из декорированных функций 📤
В декораторе можно описать, что делать со значением, возвращаемым декорированной функцией:

In [ ]:
@do_twice
def return_greeting(name):
    print("Готовлюсь приветствовать...")
    return f"Привет, {name}!"

Попытаемся использовать декорированную функцию:

In [ ]:
hi_adam = return_greeting("Адам")

In [ ]:
print(hi_adam)

К сожалению, декоратор «съел» значение, возвращаемое оригинальной функцией. Поскольку `do_twice_wrapper()` в явном виде не возвращает никакое значение, вызов в `return_greeting("Адам")` в конечном итоге вернул `None`.

Сделаем так, чтобы внутренняя функция декоратора возвращала значение декорированной функции. Поправим декоратор:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def return_greeting(name):
    print("Готовлюсь приветствовать...")
    return f"Привет, {name}!"

Проверим, как всё работает теперь:

In [ ]:
return_greeting("Адам")

## 2.6. Интроспекция: «кто ты такой, в самом деле?» 🕵

Большое удобство в работе с Python – его способность к [интроспекции](https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D1%82%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%86%D0%B8%D1%8F_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5)). У объекта есть доступ к собственным атрибутам. К примеру, у функции можно спросить её имя и вызвать документацию:

In [ ]:
print

In [ ]:
print.__name__

In [ ]:
help(print)

Интроспекция работает и для пользовательских функций:

In [ ]:
say_whee

In [ ]:
say_whee.__name__

In [ ]:
help(say_whee)

Как видим, в результате декорирования функция `say_whee()` запуталась в собственной идентичности. В результате декорирования мы получаем информацию о внутренней функции декоратора, а не об оригинальной функции. Хотя это технически верно, эта информация не очень полезна.

Чтобы исправить ситуацию, декоратор должен использовать... *специальный декоратор* `@functools.wraps`. Этот декоратор позволяет сохранить информацию об исходной функции. Вновь перепишем наш декоратор:

In [ ]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Ура!")

В самой декорируемой функции ничего менять не придется:

In [ ]:
say_whee

In [ ]:
say_whee.__name__

In [ ]:
help(say_whee)

Гораздо лучше! Теперь у функции `say_whee()` не наступает амнезии после декорирования.

# 3. Несколько примеров из реального мира🎈
Посмотрим на несколько полезных примеров декораторов. Как вы заметите, схема применения декораторов будет соответствовать тому паттерну, что мы получили в результате наших рассуждений:

```
def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Что-то делаем до
        value = func(*args, **kwargs)
        # Что-то делаем после
        return value
    return wrapper_decorator
```

Этот блок кода является хорошим шаблоном для создания более сложных декораторов.

## 3.1. Декоратор для тайминга кода ⌚
Начнем с создания декоратора `@timer`. Он будет измерять время выполнения функции и выводить результат в консоль:

In [ ]:
import time

def timer(func):
    """Выводит время выполнения декорируемой функции"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter() 
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Функция {func.__name__!r} выполнена за {run_time:.4f} с")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

Декоратор сохраняет текущее время в переменной `start_time` непосредственно перед запуском декорируемой функции. Это значение впоследствии вычитается из текущего значения `end_time` после выполнения функции. Полученная разность `run_time` передается в форматированную строку. Пара примеров:

In [ ]:
waste_some_time(1)

In [ ]:
waste_some_time(999)

**Примечание**: декоратор `@timer` отлично подходит, если вы хотите получить представление о времени выполнения функции. Для более точных замеров используйте модуль стандартной библиотеки `timeit`. Мы рассказывали о нём в Jupyter-блокноте [Назад в будущее: практическое руководство по путешествию во времени с Python](https://github.com/matyushkin/lessons/blob/master/time/Time-related%20modules%20in%20python.ipynb).

## 3.2. Отладочный декоратор 🐞🕵
Следующий декоратор `@debug` будет выводить аргументы, с которыми вызвана функция, а также возвращаемое функцией значение:

In [ ]:
def debug(func):
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Вызываем {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} возвращает {value!r}")         # 4
        return value
    return wrapper_debug

Отмеченные комментариями строки соответствуют следующим операциям:

1. Создание списка позиционных аргументов: `repr()` используется для строкового представления каждого аргумента.
2. Создание списка аргументов, передающихся по ключу: `f`-строка форматирует каждый элемент в формате `key=value` со спецификатором `!r`, соответствующим `repr()`.
3. Списки аргументов объединяются в общую подпись, элементы разделены запятыми.
4. Возвращаемое значение выводится после исполняемой функции.

Давайте посмотрим, как декоратор работает на практике, применив его к простой функции с одним позиционным аргументов и одним аргументом, передаваемым по ключу:

In [ ]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Привет, {name}!"
    else:
        return f"Ого, {name}! Тебе уже {age}, как ты быстро растёшь!"

In [ ]:
make_greeting("Бенджамин")

In [ ]:
make_greeting("Ричард", age=112)

Пример не сразу покажется полезным – декоратор `@debug` просто повторяет то, что мы ему прислали. Но он гораздо эффективнее, если его применить к небольшим вспомогательным функциям. Следующий пример иллюстрирует аппроксимацию для нахождения числа `e`.

In [ ]:
import math
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

Этот пример показывает, как вы можете применить декоратор к уже определенной функции. Аппроксимация нахождения числа $е$ основана на разложении в ряд через суммую обратных факториалов. При вызове функции `approximate_e()` вы увидим `@debug` за работой:

In [ ]:
approximate_e(5)

Видно, что сложив только пять первых членов ряда, мы получаем довольно близкое значение к числу $e$.

## 3.3. Замедление кода 🐌
Следующий пример вряд ли покажется полезным. Зачем нам вообще замедлять код Python? Например, мы хотим ограничить частоту, с которой функция проверяет обновление веб-ресурса. Декоратор `@slow_down` будет выжидать одну секунду перед запуском декорируемой функции:

In [ ]:
def slow_down(func):
    """Ждёт 1 секунду, прежде чем вызвать переданную функцию"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Поехали!")
    else:
        print(from_number)
        countdown(from_number - 1)

Чтобы увидеть результат действия декоратора, запустите пример:

In [ ]:
countdown(3)

Декоратор `@slow_down` спит всегда лишь одну секунду. Позднее мы увидим, как передавать декоратору аргумент, чтобы контролировать его скорость.

## 3.4. Регистрация плагинов
Вообще декораторы не обязаны «оборачивать» функцию, которую они декорируют. Они могут просто регистрировать то, что функция существует и возвращать на нее ссылку. Это может использоваться для создания легковесной архитектуры:

In [ ]:
import random
PLUGINS = dict()

def register(func):
    """Регистрирует функцию как плагин"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Привет, {name}!"

@register
def be_awesome(name):
    return f"Привет, {name}, классно быть вместе!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Используется {greeter!r}")
    return greeter_func(name)

В приведенном примере декоратор `@register` просто добавляет ссылку на декорируемую функцию в глобальный словарь `PLUGINS`. Никакой внутренней функции у декоратора нет, оригинальная функция возвращается немодифицированной, поэтому нет необходимости использовать `@functools.wraps`.

Функция `randomly_greet()` случайным образом выбирает, какую из зарегистрированных функций использовать для поздравления. Удобство состоит в том, что словарь `PLUGINS` уже содержит ссылку для каждой функции, к которой был применен декоратор `@register`:

In [ ]:
PLUGINS

In [ ]:
randomly_greet("Лео")

# 4. Декораторы поинтереснее 👑
До сих пор мы видели довольно простые декораторы – нам нужно было понять, как они работают. Вы можете передохнуть и попрактиковаться в применении декораторов, чтобы позднее вернуться к этому разделу, посвященному продвинутым концепциям.

## 4.1. Декорирование классов
Есть два способа применения декораторов к классам. Первый способ похож на то, что мы делали с функциями, – **декорировать методы класса**. Давайте определим класс, в котором декорируем некоторые из методов с помощью вышеописанных декораторов `@debug` и `@timer`:

In [ ]:
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Воспользуемся классом, чтобы увидеть действие декораторов:

In [ ]:
tw = TimeWaster(1000)

In [ ]:
tw.waste_time(999)

Другой подход – **декорировать классы целиком**. Написание декоратора класса очень похоже на написание декоратора функции. Разница лишь в том, что декоратор в качестве аргумента получит класс, а не функцию. Однако когда мы применяем декораторы функций к классам, их эффект может оказаться не таким, как предполагалось. В следующем примере мы применили декоратор `@timer` к классу:

In [ ]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Декорирование класса не приведет к декорированию его методов. В результате `@timer` измерит только время создания экземпляра класса:

In [ ]:
tw = TimeWaster(1000)

In [ ]:
tw.waste_time(999)

Позднее мы покажем примеры правильного декорирования классов.

## 4.2. Вложенные декораторы 🎊
К функции можно применить несколько декораторов, накладывая их действие друг на друга:

In [ ]:
@debug
@do_twice
def greet(name):
    print(f"Привет, {name}!")

В этом случае к функции будет применен сначала декоратор `@do_twice`, потом `@debug`:

In [ ]:
greet("Ева")

Посмотрим, что будет, если поменять порядок вызова декораторов:

In [ ]:
@do_twice
@debug
def greet(name):
    print(f"Привет, {name}!")

In [ ]:
greet("Ева")

## 4.3. Декораторы, принимающие аргументы 📬
Иногда полезно передавать декораторам аргументы, чтобы управлять их поведением. Например, `@do_twice` может быть расширен до декоратора `@repeat(num_times)`. Число повторений декорируемой функции можно было бы указать в качестве аргумента `num_times`.

Подумаем, как добиться такого поведения. Обычно декоратор создает и возвращает внутреннюю функцию-обертку. Мы могли бы дополнительно «обернуть» ее поведение с помощью другой внутренней функции:

In [ ]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

Немного похоже на [фильм Кристофера Нолана «Начало»](https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%87%D0%B0%D0%BB%D0%BE_(%D1%84%D0%B8%D0%BB%D1%8C%D0%BC,_2010)), но мы просто поместили один шаблон многократно выполняющего функцию декоратора в другой декоратор и добавили обработку значения аргумента.

Давайте проверим, работает ли, как задумано:

In [ ]:
@repeat(num_times=4)
def greet(name):
    print(f"Привет, {name}!")

In [ ]:
greet("мир")

## 4.4. «И того, и другого, и можно без хлеба!»
Немного потрудившись, мы можем определить декоратор, который можно использовать как с аргументами, так и без них.

Поскольку ссылка на декорируемую функцию передается напрямую только в случае, если декоратор был вызван без аргументов, ссылка на функцию должна быть необязательным аргументом. То есть все аргументы декоратора должны передаваться по ключу. Для этого мы можем применить специальный синтаксис (`*`), указывающий, что все остальные аргументы передаются по ключу:

```
def name(_func=None, *, kw1=val1, kw2=val2, ...):
    def decorator_name(func):
        ...  # Создает и возвращает функцию-обёртку.

    if _func is None:
        return decorator_name
    else:
        return decorator_name(_func) 
```

Здесь аргумент `_func` действует как маркер, отмечающий, был ли декоратор вызван с аргументами или без них.

Если функция декоратора `name` будет вызвана без аргументов, декорируемая функция будет передана как `_func`. Если декоратор будет вызван с аргументами, тогда значение `_func` останется `None`, а передаваемые по ключу аргументы заменят значения по умолчанию. Символ `*` в списке аргументов означает, что следующие за ним аргументы не могут быть переданы как позиционные.

То есть в сравнении с предыдущей версией к декоратору добавилось условие `if-else`:

In [ ]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

In [ ]:
@repeat
def say_whee():
    print("Ура!")

@repeat(num_times=3)
def greet(name):
    print(f"Привет, {name}!")

In [ ]:
say_whee()

In [ ]:
greet('мир')

## 4.5. Декораторы, хранящие состояние 💾
Иногда полезно иметь декораторы, отслеживающие состояние. В качестве простого примера мы создадим декоратор, который подсчитывает, сколько раз вызывалась функция.

В следующем разделе мы увидим, как использовать для сохранения состояния классы. Но в простых случаях достаточно декораторов функций:

In [ ]:
def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"{wrapper_count_calls.num_calls} вызов функции {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Ура!")

In [ ]:
say_whee()

In [ ]:
say_whee()

In [ ]:
say_whee.num_calls

## 4.6. Классы в качестве декораторов функций
Обычным способом хранения состояния является использование классов. Перепишем `@count_calls` из предыдущего раздела, используя в качестве декоратора класс.

Напомним, что синтаксис декоратора @my_decorator – это всего лишь более простой способ сказать `func = my_decorator(func`). Если my_decorator является классом, он должен принять func в качестве аргумента в методе `__init__()`.

Кроме того, класс должен быть вызван так, чтобы он его можно было вызвать вместо декорируемой функции. Для этого в нем должен быть описан метод `__call__()`:

In [ ]:
class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f"Текущее значение счетчика – {self.count}")

Метод `__call__()` вызывается всякий раз, когда мы обращаемся к экземпляру класса:

In [ ]:
counter = Counter()

In [ ]:
counter()

In [ ]:
counter()

In [ ]:
counter.count

Таким, образом типичная реализация класса декоратора должна содержать `__init__()` и `__call__()`:

In [ ]:
class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"{self.num_calls} вызов функции {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Ура!")

Метод `__init__()` должен хранить ссылку на функцию и может выполнять любую другую необходимую инициализацию.

Метод `__call__()` будет вызываться вместо декорированной функции. По сути, он делает то же самое, что и функция wrapper() в наших предыдущих примерах.

Обратите внимание, что в случае методов классов нужно использовать функцию `functools.update_wrapper()` вместо `@functools.wraps`.

Декоратор `@CountCalls` работает так же, как и в предыдущем разделе:

In [ ]:
say_whee()

In [ ]:
say_whee()

In [ ]:
say_whee.num_calls

# 5. Ещё несколько примеров из реального мира 🧭
Мы прошли долгий путь и узнали, как создаются всевозможные декораторы. Давайте подведем итоги, применив полученные знания для анализа полезных на практике программных конструкций.

## 5.1. Вновь замедляем код, но уже по-умному 🐢
Наша предыдущая реализация замедлителя кода `@slow_down` всегда «усыпляла» декорируемую функцию на одно и то же время. Давайте воспользуемся нашими знаниями о передачи в декоратор аргументов:

In [ ]:
def slow_down(_func=None, *, rate=1):
    """Усыпляет функцию перед вызовом на переданное количество секунд"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

Проверим на примере функции `countdown()`:

In [ ]:
@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Поехали!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [ ]:
countdown(3)

## 5.2. Создание синглтонов 🗿
Синглтон – это класс с единственным экземпляром. В Python есть несколько часто используемых синглтонов, к примеру: `None`, `True` и `False`. Тот факт, что `None` является синглтоном, позволяет использовать оператор `is` для сравнения объектов с `None`. Мы пользовались этим выше:

```
if _func is None:
    return decorator_name
else:
    return decorator_name(_func)
```

Оператор `is` возвращает `True` только для объектов, представляющих одну и ту же сущность.

Описанный ниже декоратор `@singleton` превращает класс в одноэлементный, сохраняя первый экземпляр класса в качестве атрибута. Последующие попытки создания экземпляра просто возвращают сохраненный экземпляр:

In [ ]:
def singleton(cls):
    """Превращает класс в Singleton-класс с единственным экземпляром"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

Как видите, этот декоратор класса следует тому же шаблону, что и наши декораторы функций. Единственное отличие состоит в том, что мы используем `cls` вместо `func` в качестве имени параметра.

In [ ]:
first_one = TheOne()
another_one = TheOne()

In [ ]:
id(first_one)

In [ ]:
id(another_one)

In [ ]:
first_one is another_one

Таким образом, `first_one` действительно представляет тот же экземпляр, что и `another_one`.

**Примечание**. Singleton-классы не так часто используются в Python, как в других языках. Эффект синглтона обычно лучше реализовать через глобальную переменную модуля.

## 5.3. Кэширование возвращаемых значений ⏳
Декораторы предоставляют прекрасный механизм для кэширования и мемоизации. В качестве примера давайте рассмотрим рекурсивное определение последовательности Фибоначчи:

In [ ]:
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

Хотя реализация и выглядит просто, с производительностью дела обстоят плохо:

In [ ]:
fibonacci(10)

In [ ]:
fibonacci.num_calls

Чтобы рассчитать десятое число в последовательности Фибоначчи, в действительности достаточно лишь вычислить предыдущие числа этого ряда. Однако указанная реализация требует выполнения `177` вычислений. И ситуация быстро ухудшается: для 30-го числа потребуется 2.7 млн. операций. Это объясняется тем, что код каждый раз пересчитывает числа последовательности, уже известные из предыдущих этапов.

Обычное решение состоит в том, чтобы находить числа Фибоначчи, используя цикл for и справочную таблицу. Тем не менее, можно просто добавить к рекурсии кэширование вычислений:

In [ ]:
def cache(func):
    """Кэширует предыдущие вызовы функции"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

Кэш работает как справочная таблица, поэтому теперь `fibonacci()` выполняет необходимые вычисления только один раз:

In [ ]:
fibonacci(10)

In [ ]:
fibonacci(8)

Заметьте, что при вызове `fibonacci(8)` не происходит никаких дополнительных расчетов – все необходимые значения уже найдены и сохранены при вычислении `fibonacci(11)`.

**Примечание**: в стандартной библиотеке Python есть также декоратор для [LRU-кэширования](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%BA%D1%8D%D1%88%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F) – [`@functools.lru_cache`](https://docs.python.org/library/functools.html#functools.lru_cache). Этот декоратор имеет больше возможностей, чем тот, что мы написали выше.

## 5.4. Добавление единиц измерения ⚖️
Следующий пример похож на задачу о регистрации плагинов (функций) – здесь тоже не будет меняться поведение декорированной функции. Вместо этого к атрибутам функции будут добавляться единицы измерения:

In [ ]:
def set_unit(unit):
    """Регистрирует юнит для переданной функции"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

В следующем примере вычисляется объем цилиндра по известному радиусу и высоте, указанных в сантиметрах:

In [ ]:
@set_unit("см^3")
def volume(radius, height):
    return math.pi * radius**2 * height

Атрибут `unit` можно далее использовать по мере необходимости:

In [ ]:
volume(3, 5)

In [ ]:
volume.unit

**Примечание**: если вам нужно регулярно использовать единицы измерения, например, преобразовывать одни единицы в другие, обратите внимание на библиотеку pint: [pip install Pint](http://pint.readthedocs.io/).

# Заключение
Поздравляем, вы дошли до конца статьи! 🎖️

Итак, теперь вы знаете:

* Как создавать декораторы функций и классов.
* Как передавать в декораторы аргументы и возвращать из них значения.
* Зачем в декораторах используется @functools.wraps.
* Как использовать вложенные декораторы.
* Как при помощи декораторов хранить состояния и кэшировать результаты функций.

В определении декораторов нет никакой магии. Обычно всё направлено на создание функции или класса, выступающих в качестве обёртки над оригинальной функцией. Для передачи аргументов применяется обычная нотация `*args` и `**kwargs`. А использование знака `@` представляет лишь синтаксический сахар, облегчающий вызов декораторов.

Декораторы очень удобны, чтобы модифицировать поведение функций и классов, создавать для их обработки дополнительную логику. При этом такие шаблоны модификации легко наслаивать друг на друга с помощью вложенных декораторов. Чтобы снять декорирование, достаточно просто удалить строчку с упоминанием декоратора.

Для ещё более глубокого погружения в декораторы, посмотрите исторический документ [PEP 318](https://www.python.org/dev/peps/pep-0318/), а также [вики-страницу](https://wiki.python.org/moin/PythonDecorators), посвященную декораторам Python.

Сторонний модуль [decorator](https://realpython.com/primer-on-python-decorators/) поможет вам в создании собственных декораторов. Его [документация](https://github.com/micheles/decorator/blob/master/docs/documentation.md) содержит ещё больше примеров использования декораторов.

Если вам понравилось содержание этого блокнота, вы можете поставить звезду и подписаться на обновления [репозитория lessons](https://github.com/matyushkin/lessons), в котором я выкладываю различные образовательные материалы, переводы и оригинальные статьи.

Мои образовательные блокноты, которые так же, как эта статья, касаются более глубоких тем в Python:
* [О модуле Collections](https://github.com/matyushkin/lessons/tree/master/collections)
* [О модуле itertools](https://github.com/matyushkin/lessons/tree/master/itertools)
* [О работе с time-библиотеками](https://github.com/matyushkin/lessons/tree/master/time)